# Tutorial

This tutorial will guide you in the basic use of ThermoState. The ThermoState
package is designed to ease the evaluation of thermodynamic properties for
common substances used in Mechanical Engineering courses. Rather than looking up
the information in a table and interpolating, we can input properties for the
states directly, and all unknown values are automatically calculated.

ThermoState uses [CoolProp](http://www.coolprop.org/) and [Pint](https://pint.readthedocs.io) to enable easy property evaluation in any unit system. The first thing we need to do is import the parts of ThermoState that we will use. This adds them to the set of local variables

In [1]:
from thermostate import State, Q_, units

## Pint and Units

Now that the interface has been imported, we can create some properties. For instance, let's say we're given the pressure and temperature properties for water, and asked to determine the specific volume. First, let's create variables that set the pressure and temperature. We will use the Pint `Quantity` function, which we have called `Q_`. The syntax for the `Q_` function is `Q_(value, 'units')`.

In [2]:
p_1 = Q_(101325, 'Pa')

We can use whatever units we'd like, Pint supports a wide variety of units.

In [12]:
p_1 = Q_(1.01325, 'bar')
p_1 = Q_(14.7, 'psi')
p_1 = Q_(1.0, 'atm')

Another way to specify the units is to use the `units` class that we imported. This class has a number of attributes (text following a period) that can be used to create a quantity with units by multiplying a number with the unit. Let's set the temperature now. The available units of temperature are `degF` (`fahrenheit`), `degR` (`rankine`), `degC` (`celsius`), and `K` (`kelvin`).

In [8]:
T_1 = 460*units.degR
T_1 = 25*units.degC
T_1 = 75*units.degF
T_1 = 400*units.K

The two ways of creating the units are equivalent

In [9]:
Q_(101325, 'Pa') == 1.0*units.atm

True

## ThermoState

Now that we have defined two properties with units, let's define a state. First, we create a variable to hold the State and tell what substance we want to use with that state. Available substances are:

* `water`
* `air`
* `R134a`
* `R22`
* `propane`
* `ammonia`.

In [10]:
st_1 = State('water')

Now that the state is created, we need to assign values for the properties. Properties of the state are accessed as attributes, and they must always be set in pairs, we cannot set a single property at a time. In this case we will access the `Tp` property of the state and set it equal to the temperature and pressure we defined above. The syntax is

    state.Tp = temperature, pressure
    
We could also use the `pT` property to accomplish the same thing

    state.pT = pressure, temperature
    
Note that the capitalization of the properties is important! The `p` is lower case while the `T` is upper case (lower case `t` means time).

<div class="alert alert-warning">

**Warning:**

Remember that two independent and intensive properties are required to set the state!

</div>

In [14]:
print('T = {}, p = {}'.format(T_1, p_1))

T = 300 kelvin, p = 1.0 atmosphere


In [15]:
st_1.Tp = T_1, p_1
st_1.pT = p_1, T_1

By setting two properties in this way, the `State` class will calculate all the other properties we might be interested in. We can access the value of any property by getting the attribute for that property. The available properties are `T` (temperature), `p` (pressure), `v` (mass-specific volume), `u` (mass-specific internal energy), `h` (mass-specific enthalpy), `s` (mass-specific entropy), and `x` (quality). The syntax is

    State.property
    
or

    st_1.T  # Gets the temperature
    st_1.p  # Gets the pressure
    st_1.v  # Gets the specific volume
    st_1.u  # Gets the internal energy
    st_1.h  # Gets the enthalpy
    st_1.s  # Gets the entropy
    st_1.x  # Gets the quality
    


<div class="alert alert-info">

**Note:**

Capitalization is important! The temperature has upper case `T`, while the other properties are lower case to indicate that they are mass-specific quantities.

</div>

In [19]:
print('T_1 = {}'.format(st_1.T))
print('p_1 = {}'.format(st_1.p))
print('v_1 = {}'.format(st_1.v))
print('u_1 = {}'.format(st_1.u))
print('h_1 = {}'.format(st_1.h))
print('s_1 = {}'.format(st_1.s))
print('x_1 = {}'.format(st_1.x))

T_1 = 300 kelvin
p_1 = 101325.0 pascal
v_1 = 0.0010034549603870667 meter ** 3 / kilogram
u_1 = 112553.2245807535 joule / kilogram
h_1 = 112654.8996546125 joule / kilogram
s_1 = 393.0620684406139 joule / kelvin / kilogram
x_1 = None


In this case, the value for the quality is the special Python value `None`. This is because at 400 K and 101325 Pa, the state of water is a **superheated vapor** and the quality is **undefined** except in the vapor dome. To access states in the vapor dome, we cannot use `T` and `p` as independent properties, because they are not independent inside the vapor dome. Instead, we have to use the pairs involving the other properties (possibly including the quality) to set the state. When we define the quality, the units are `dimensionless` or `percent`. For instance:

In [20]:
T_2 = Q_(100.0, 'degC')
x_2 = Q_(0.1, 'dimensionless')
st_2 = State('water')
st_2.Tx = T_2, x_2
print('T_2 = {}'.format(st_2.T))
print('p_2 = {}'.format(st_2.p))
print('v_2 = {}'.format(st_2.v))
print('u_2 = {}'.format(st_2.u))
print('h_2 = {}'.format(st_2.h))
print('s_2 = {}'.format(st_2.s))
print('x_2 = {}'.format(st_2.x))

T_2 = 373.15 kelvin
p_2 = 101417.99665788244 pascal
v_2 = 0.16811572834411825 meter ** 3 / kilogram
u_2 = 627756.5746698529 joule / kilogram
h_2 = 644806.535045544 joule / kilogram
s_2 = 1911.9019425021506 joule / kelvin / kilogram
x_2 = 0.1 dimensionless


In [21]:
x_2 == Q_(10.0, 'percent')

True

From these results, we can see that the units of the units of the properties stored in the `State` are always SI units - Kelvin, Pascal, m<sup>3</sup>/kg, J/kg, and J/(kg-Kelvin). We can use the `to` function to convert the units to anything we want, provided the dimensions are compatible. The syntax is `State.property.to('units')`.

In [22]:
print(st_2.T.to('degF'))
print(st_2.s.to('BTU/(lb*degR)'))

212.00000039999995 degF
0.4566499337207774 btu / degR / pound


<div class="alert alert-info">

**Note:**

The values are always converted in the State to SI units, no matter what the input units are. Therefore, if you want EE units as an output, you have to convert.

</div>

If we try to convert to a unit with incompatible dimensions, Pint will raise a `DimenstionalityError` exception.

<div class="alert alert-warning">

**Warning:**

If you get a `DimensionalityError`, examine your conversion very closely. The error message will tell you why the dimensions are incompatible!

</div>

In [23]:
print(st_2.T.to('joule'))

DimensionalityError: Cannot convert from 'kelvin' ([temperature]) to 'joule' ([length] ** 2 * [mass] / [time] ** 2)

Here we have tried to convert from `'kelvin'` to `'joule'` and the error message **which is the last line** says
    
    DimensionalityError: Cannot convert from 'kelvin' ([temperature]) to 'joule' ([length] ** 2 * [mass] / [time] ** 2)
    
The dimensions of a temperature are, well, temperature. The formula for energy (Joule) is $m*a*d$ (mass times acceleration times distance), and in terms of dimensions, $M*L/T^2*L = L^2*M/T^2$ (where in dimensions, capital $T$ is time). Clearly, these dimensions are incompatible. A more subtle case might be trying to convert **energy** to **power** (again, not allowed):

In [24]:
Q_(1000.0, 'joule').to('watt')

DimensionalityError: Cannot convert from 'joule' ([length] ** 2 * [mass] / [time] ** 2) to 'watt' ([length] ** 2 * [mass] / [time] ** 3)

## Summary

In summary, we need to use two (2) **independent and intensive** properties to fix the state of any simple compressible system. We need to define these quantities with units using Pint, and then use them to set the conditions of a State. Then, we can access the other properties of the State by using the attributes.

In [30]:
h_3 = Q_(2000.0, 'kJ/kg')
s_3 = Q_(3.10, 'kJ/(kg*K)')
st_3 = State('water')
st_3.hs = h_3, s_3
print('T_3 = {}'.format(st_3.T))
print('p_3 = {}'.format(st_3.p))
print('v_3 = {}'.format(st_3.v))
print('u_3 = {}'.format(st_3.u))
print('h_3 = {}'.format(st_3.h))
print('s_3 = {}'.format(st_3.s))
print('x_3 = {}'.format(st_3.x))

T_3 = 666.0536816976213 kelvin
p_3 = 669560197.2594368 pascal
v_3 = 0.0010136928689306822 meter ** 3 / kilogram
u_3 = 1321271.6027173032 joule / kilogram
h_3 = 2000000.0 joule / kilogram
s_3 = 3100.0 joule / kelvin / kilogram
x_3 = None
